# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?** **-->** Answer: **8.382332347441762**
- **Question 2: What is the 13th number yielded** --> Answer **3.605551275463989**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [49]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

total = 0
for idx, sqrt_value in enumerate(generator, 1):
    total += sqrt_value
    print(f"{idx}: {sqrt_value}, {total=}")


1: 1.0, total=1.0
2: 1.4142135623730951, total=2.414213562373095
3: 1.7320508075688772, total=4.146264369941973
4: 2.0, total=6.146264369941973
5: 2.23606797749979, total=8.382332347441762
6: 2.449489742783178, total=10.83182209022494
7: 2.6457513110645907, total=13.47757340128953
8: 2.8284271247461903, total=16.30600052603572
9: 3.0, total=19.30600052603572
10: 3.1622776601683795, total=22.4682781862041
11: 3.3166247903554, total=25.7849029765595
12: 3.4641016151377544, total=29.249004591697254
13: 3.605551275463989, total=32.854555867161245


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [8]:
import dlt
import duckdb

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

# define the connection to load to.
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')
people_1_loader = generators_pipeline.run(
    people_1(),
    table_name="people",
    write_disposition="replace",
)
print(people_1_loader)

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')
people_2_loader = generators_pipeline.run(
    people_2(),
    table_name="people",
    write_disposition="append",
)
print(people_2_loader)
print(generators_pipeline.pipeline_name)

# Check data and get sum of ages
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")

people = conn.sql(
"""
SELECT *
FROM people p
ORDER BY p.name
"""
).df()
display(people)

# TASK
# calculate the sum of all ages of people.

total_age_sum = conn.sql(
"""
SELECT SUM(p.age)
FROM people p
"""
).df()
conn.close()
print(f"{total_age_sum}")

# After correctly appending the data, calculate the sum of all ages of people.
353.0



Pipeline dlt_ipykernel_launcher load step completed in 0.21 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////home/tom/projects/de-zoomcamp/homework/03-data-ingestion-dlt-workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707339561.8521063 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.20 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////home/tom/projects/de-zoomcamp/homework/03-data-ingestion-dlt-workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707339562.454879 is LOADED and contains no failed jobs
dlt_ipykernel_launcher


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707339561.8521063,lGgpOyxz/vjAew,None
1,2,Person_2,27,City_A,1707339561.8521063,3s0Egqt1itkKOg,None
2,3,Person_3,28,City_A,1707339561.8521063,vzHyeYvwboeCFg,None
3,3,Person_3,33,City_B,1707339562.454879,JHfU6chDahsrFQ,Job_3
4,4,Person_4,29,City_A,1707339561.8521063,zr0aW7pUzszLZA,None
5,4,Person_4,34,City_B,1707339562.454879,aYz49rHMrWXgzg,Job_4
6,5,Person_5,30,City_A,1707339561.8521063,IL1508wIENxT/A,None
7,5,Person_5,35,City_B,1707339562.454879,FYfmNBHvk3lu6A,Job_5
8,6,Person_6,36,City_B,1707339562.454879,2aY3yxhxjfOQ5g,Job_6
9,7,Person_7,37,City_B,1707339562.454879,ws1kEK4/o87f3A,Job_7


   sum(p.age)
0       353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [52]:
#Install the dependencies
# %%capture
# !pip install dlt[duckdb]

UsageError: Line magic function `%%capture` not found.


In [10]:
import dlt
import duckdb

# Generators
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')
people_1_loader = generators_pipeline.run(
    people_1(),
    table_name="people_merge",
    write_disposition="replace",
)
print(people_1_loader)


generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')
people_2_loader = generators_pipeline.run(
    people_2(),
    table_name="people_merge",
    write_disposition="merge",
    primary_key="ID",
)
print(people_2_loader)
print(generators_pipeline.pipeline_name)

# Check data and get sum of ages
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")

people = conn.sql(
"""
SELECT *
FROM people_merge pm
ORDER BY pm.name
"""
).df()
display(people)

# TASK
# calculate the sum of all ages of people.

total_age_sum = conn.sql(
"""
SELECT SUM(pm.age)
FROM people_merge pm
"""
).df()
conn.close()
print(f"{total_age_sum}")

# Calculate the sum of ages of all the people loaded as described above.
266.0


Pipeline dlt_ipykernel_launcher load step completed in 0.27 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////home/tom/projects/de-zoomcamp/homework/03-data-ingestion-dlt-workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707339949.5313067 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.35 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////home/tom/projects/de-zoomcamp/homework/03-data-ingestion-dlt-workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707339950.218018 is LOADED and contains no failed jobs
dlt_ipykernel_launcher


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707339949.5313067,zePnCc94DVwePw,None
1,2,Person_2,27,City_A,1707339949.5313067,4fOXKMeXfWm6cQ,None
2,3,Person_3,33,City_B,1707339950.218018,i0NLeItwB+RMcQ,Job_3
3,4,Person_4,34,City_B,1707339950.218018,LvdbZUg0un+7EQ,Job_4
4,5,Person_5,35,City_B,1707339950.218018,WSn40bwvFTEKxg,Job_5
5,6,Person_6,36,City_B,1707339950.218018,+CXM3wniVvvLQA,Job_6
6,7,Person_7,37,City_B,1707339950.218018,LnvQubNFLOXgwg,Job_7
7,8,Person_8,38,City_B,1707339950.218018,IHughwTdhdpBQA,Job_8


   sum(pm.age)
0        266.0


266.0

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX